In [ ]:
!pip install keras-retinanet==0.5 tensorflow-gpu==1.15.2 keras==2.3.1

In [22]:
import os
import numpy as np

In [23]:
import tensorflow as tf

In [24]:
from keras_retinanet import models
from keras_retinanet.utils.image import (read_image_bgr, preprocess_image, resize_image)
from keras_retinanet.utils.visualization import (draw_box, draw_caption)
from keras_retinanet.utils.colors import label_color

In [25]:
from PIL import Image

In [30]:
CAR_LABEL = {
    0: 'car',
    1: 'truck',
    2: 'bus',
}


def _make_mask(height, width):
    return np.ones((height, width), dtype=np.uint8)*255


def _mask_car(img, box):
    img[box[1]:box[3], box[0]:box[2]] = 0
    return img


def _detect_car(model_path, working_dir, images):
    # load the model
    model = models.load_model(model_path, backbone_name='resnet50')

    # for the processing result
    info = dict()

    os.makedirs(os.path.join(working_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(working_dir, 'cars_detected'), exist_ok=True)
    os.makedirs(os.path.join(working_dir, 'masks'), exist_ok=True)
    
    for filename in images:
        info[filename] = dict()

        image_path = os.path.join(working_dir, 'images', filename)
        result_path = os.path.join(working_dir, 'cars_detected', filename)
        mask_path = os.path.join(working_dir, 'masks', os.path.splitext(filename)[0] + '.png')

        # Load images
        im = read_image_bgr(image_path)

        # Make masks
        height, width = im.shape[0:2]
        mask = _make_mask(height, width)
        info[filename]['shape'] = (height, width)

        # Prepare images to draw on (bgr -> rgb)
        im_labeled = im[:, :, ::-1].copy()

        # Preprocess images for network
        im = preprocess_image(im)
        im, scale = resize_image(im, min_side=2600, max_side=99999)

        # Process images
        boxes, scores, labels = model.predict_on_batch(
            np.expand_dims(im, axis=0))

        # Correct for image scales
        boxes /= scale

        box_list = []
        for i, (box, score, label) in enumerate(zip(boxes[0], scores[0], labels[0])):
            # Scores are sorted so we can break
            if score < 0.5:
                break

            if label in CAR_LABEL:
                box = box.astype(int)

                color = label_color(label)
                draw_box(im_labeled, box, color=color)
                caption = "{} {} {:.3f}".format(i, CAR_LABEL[label], score)
                draw_caption(im_labeled, box, caption)

                mask = _mask_car(mask, box)

                box_list.append(box.tolist())

        # Save images
        im = Image.fromarray(im_labeled)
        im.save(result_path)
        im = Image.fromarray(mask)
        im.save(mask_path)

        info[filename]['boxes'] = box_list

    return info

In [31]:
import os

info = _detect_car(r'resnet50_model.h5', r'Test', os.listdir(os.path.join(r'Test', r'images')))

tracking <tf.Variable 'Variable_35:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_36:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_37:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_38:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_39:0' shape=(9, 4) dtype=float32> anchors


In [32]:
info

{'1.jpg': {'shape': (449, 800), 'boxes': [[149, 138, 247, 189]]},
 '2.jpg': {'shape': (956, 1300),
  'boxes': [[742, 229, 805, 279],
   [727, 335, 793, 370],
   [668, 281, 734, 330],
   [1022, 519, 1097, 568],
   [623, 499, 696, 528],
   [644, 557, 720, 587],
   [1024, 542, 1094, 567],
   [1024, 542, 1094, 567]]}}